In [1]:
import numpy as np
import pandas as pd

In [42]:
train_df= pd.read_csv('train.csv')

In [43]:
topic_cluster_mapping = {
    "economic": [
        'Economy',
        'Economy.Bank and Bourse',
        'Economy.Oil',
        'Economy.Commerce',
        'Economy.Industry',
        'Economy.Agriculture',
        'Economy.Dwelling and Construction'
    ],
    "cultural and artistic": [
        'Science and Culture',
        'Science and Culture.Science.Book',
        'Literature and Art',
        'Literature and Art.Art',
        'Literature and Art.Art.Cinema',
        'Literature and Art.Art.Music',
        'Literature and Art.Art.Theater',
        'Science and Culture.Science',
        'Science and Culture.Science.Medicine and Remedy',
        'Literature and Art.Literature'
    ],
    "political": [
        'Politics',
        'Politics.Iran Politics'
    ],
    "technological": [
        'Science and Culture.Science.Information and Communication Technology'
    ],
    "sports": [
        'Sport',
        'Sport.World Cup'
    ],
    "social": [
        'Social',
        'Social.Women',
        'Social.Religion'
    ],
    "miscellaneous": [
        'Miscellaneous',
        'Miscellaneous.World News',
        'Miscellaneous.Urban',
        'Miscellaneous.Picture',
        'Miscellaneous.Happenings',
        'Miscellaneous.Islamic Councils',
        'Miscellaneous.Picture.Caricature'
    ],
    "tourism": [
        'Tourism'
    ],
    "environment": [
        'Natural Environment'
    ]
}

def categoesMapping(cat):
    for key,value in topic_cluster_mapping.items():
        if cat in value:
            return key

In [44]:
categiesdData=train_df['Category'].apply(categoesMapping)


In [45]:
categiesdData

0         cultural and artistic
1                        sports
2                      economic
3                 miscellaneous
4                        sports
                  ...          
150091            miscellaneous
150092            miscellaneous
150093            miscellaneous
150094    cultural and artistic
150095                political
Name: Category, Length: 150096, dtype: object

In [46]:
from sklearn import preprocessing 
  
# label_encoder object knows  
# how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
categiesdData= label_encoder.fit_transform(categiesdData) 
  


In [47]:
import re

# Define a regex pattern to match URLs
url_pattern = re.compile(r'https?://\S+')

# Define a function to remove URLs from text
def remove_urls(text):
    return url_pattern.sub('', text)

# Apply the function to the 'text' column and create a new column 'clean_text'
train_df['Text'] = train_df['Text'].apply(remove_urls)

train_df['Text'] = train_df['Text'].replace(to_replace=r'[^\w\s]', value='', regex=True)

train_df['Text'] = train_df['Text'].replace(to_replace=r'\d', value='', regex=True)

In [50]:
train_df['Text'] = train_df['Text'].replace(to_replace=r'\n', value='', regex=True)

In [51]:
processedData=train_df['Text']

In [53]:
from hazm import word_tokenize
# converting each sentence to list of words and inserting in sents
processedData= processedData.apply(word_tokenize)

In [54]:
processedData.head()

0    [خبرنامه, دانشگاه, علم, و, صنعت, ايران, شماره,...
1    [تا, پايان, سال, دهها, زمين, فوتبال, و, سالن, ...
2    [انجمن, توليدكنندگان, تجهيزات, صنعت, نفت, تشكي...
3    [كرتين, براي, سومين, بار, نخست, وزير, كانادا, ...
4    [خداحافظ, رفقا, نمايندگان, اروپاي, شرقي, در, ج...
Name: Text, dtype: object

In [56]:
pd.DataFrame(processedData).to_csv('processedText.csv',header=None,index=None)

In [57]:
stopWordsFile = open('./stopwords.dat')
stopWordsFile2 = open('./persian-stopwords/persian')
combined=stopWordsFile.read().split('\n') + stopWordsFile2.read().split('\n')
stopWords= set(combined)

In [58]:
processedData=processedData.apply(lambda x: [word for word in x if word not in stopWords])

In [59]:
pd.DataFrame(processedData).to_csv('finalTexts.csv',header=None,index=None)


In [64]:
df = pd.read_csv('finalTexts.csv',header=None)

In [95]:
texts=df[0].apply(lambda text : eval(text))


0         [خبرنامه, دانشگاه, علم, صنعت, ايران, شماره, يا...
1         [پايان, سال, دهها, زمين, فوتبال, سالن, ورزش, ب...
2         [انجمن, توليدكنندگان, تجهيزات, صنعت, نفت, تشكي...
3         [كرتين, براي, سومين, وزير, كانادا, ژان, كرتين,...
4         [رفقا, نمايندگان, اروپاي, شرقي, جام, بابك, كما...
                                ...                        
150091    [مشكل, سدسازي, وزير, نيرو, كشور, كمبود, اعتبار...
150092    [خودسوزي, يك, هوادار, ك, ك, شهر, دياربكر, تركي...
150093    [نيمه, سال, جاري, دستگاه, اتوبوس, كارت, بليت, ...
150094    [كتابخانه, مركز, دانشگاه, تهران, دانشكده, هنره...
150095    [پيام, رئيس, جمهوري, سران, كشورهاي, عضو, سازما...
Name: 0, Length: 150096, dtype: object

In [117]:
texts.shape

(150096,)

In [118]:
texts.to_csv('textesList.csv',header=None,index=None)

In [166]:
from sklearn.model_selection import train_test_split
X_train , x_test ,y_train ,y_test = train_test_split(texts,categiesdData,test_size=0.2,random_state=42)


In [165]:
VECTOR_SIZE,ZARP = 50,30

50

In [167]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(X_train, vector_size=VECTOR_SIZE, window=5, min_count=5, workers=4)

In [168]:
def vectorize(sentence):
    words_vecs = [w2v_model.wv[word] for word in sentence if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(VECTOR_SIZE)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train = np.array([vectorize(sentence) for sentence in X_train])

In [169]:
x_test = np.array([vectorize(sentence) for sentence in x_test])

In [170]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(x_test)


c:\Users\Eniac\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [172]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=15,min_samples_leaf=5)
rfc.fit(X_train, y_train)
rfc_y_pred = clf.predict(x_test)


In [ ]:
dictofAccuracyVectorSize = {
    {'VECTOR_SIZE': 24,   'Accuracy': 0.724666326145373}
}

In [173]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy:', accuracy_score(y_test, rfc_y_pred))
print('Precision:', precision_score(y_test, rfc_y_pred,average='weighted'))
print('Recall:', recall_score(y_test, rfc_y_pred,average='weighted'))
print('F1 score:', f1_score(y_test, rfc_y_pred,average='weighted'))

Accuracy: 0.7427048634243838
Precision: 0.7346016550281594
Recall: 0.7427048634243838
F1 score: 0.7339464886130384


In [174]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred,average='micro'))
print('Recall:', recall_score(y_test, y_pred,average='micro'))
print('F1 score:', f1_score(y_test, y_pred,average='weighted'))

Accuracy: 0.7427048634243838
Precision: 0.7427048634243838
Recall: 0.7427048634243838
F1 score: 0.7339464886130384


In [11]:
X_train

0       [خبرنامه, دانشگاه, علم, صنعت, ايران, شماره, يا...
1       [پايان, سال, دهها, زمين, فوتبال, سالن, ورزش, ب...
2       [انجمن, توليدكنندگان, تجهيزات, صنعت, نفت, تشكي...
3       [كرتين, براي, سومين, وزير, كانادا, ژان, كرتين,...
4       [رفقا, نمايندگان, اروپاي, شرقي, جام, بابك, كما...
                              ...                        
9995    [مسئوليت, شهروندان, رئيس, جمهور, راي, خويش, تي...
9996    [مديرمسئول, روزنامه, همبستگي, دادگاه, احضار, گ...
9997    [شيرهاي, آب, توجه, موقعيت, بحراني, ذخاير, آب, ...
9998    [نيم, رخ, نگاهي, زندگي, جبران, خليل, جبران, پس...
9999    [پيروزي, پيكان, مدعي, واليبال, برتر, باشگاههاي...
Name: Text, Length: 10000, dtype: object

In [12]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(sentences=X_train, vector_size=50, window=5,seed=42, min_count=10, workers=4)

In [144]:
w2v_model.wv.most_similar('اقتصاد',topn=10)

[('ليگ', 0.9693446755409241),
 ('باشگاهي', 0.9688852429389954),
 ('بازيهاي', 0.9644466042518616),
 ('قهرماني', 0.9510931372642517),
 ('تيمهاي', 0.9510273337364197),
 ('تيمي', 0.9507378339767456),
 ('رقابتهاي', 0.9420532584190369),
 ('برتر', 0.9419983625411987),
 ('بازيكنانش', 0.9363963007926941),
 ('گيران', 0.9300103187561035)]

In [13]:
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

In [14]:
X_train = np.array([vectorize(sentence) for sentence in X_train])
